In [ ]:
#from threads_to_doc_list import threads_to_doc_list

In [4]:
filename = '/kaggle/input/reddit-worldcup/reddit_worldcup_data_500_weekly2022-11-21 11_56_41.json'

#doc_list = threads_to_doc_list(filename)#, 1668988801)
doc_list = threads_to_exploded_doc_list(filename)

In [7]:
from stop_words import get_stop_words
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
import pandas as pd
import gensim
import pyLDAvis.gensim
import nltk

nltk.download('omw-1.4')

/opt/conda/lib/python3.7/site-packages/past/types/oldstr.py:36: DeprecationWarning: invalid escape sequence \d
  """
/opt/conda/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [8]:
drop_word_list = ['www', 'http', 'com', 'country', 'world', 'cup']
pattern = r'\b[^\d\W]+\b'
tokenizer = RegexpTokenizer(pattern)
en_stop_and_drop_list = get_stop_words('en') + drop_word_list
lemmatizer = WordNetLemmatizer()


In [9]:
def preprocess_doc(doc):
    doc = doc.lower()
    tokenized_doc = tokenizer.tokenize(doc)
    no_stop_word_tokens = [token for token in tokenized_doc if not token in en_stop_and_drop_list]
    lemma_tokens = [lemmatizer.lemmatize(token) for token in no_stop_word_tokens]
    processed_doc = [token for token in lemma_tokens if not len(token) == 1]
    return processed_doc

def preprossess_doc_list(doc_list):
    clean_doc_list = []
    for d in doc_list:
        clean_doc_list.append(preprocess_doc(d))
    return clean_doc_list
        

In [10]:
processed_doc_list = preprossess_doc_list(doc_list)

In [11]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(processed_doc_list)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(doc) for doc in processed_doc_list]

In [14]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=20)
import pprint
pprint.pprint(ldamodel.top_topics(corpus,topn=5))

[([(0.015524723, 'team'),
   (0.013851552, 'can'),
   (0.013770097, 'fan'),
   (0.012780823, 'game'),
   (0.012342569, 'qatar')],
  -1.9991432783660037),
 ([(0.019218042, 'qatar'),
   (0.014981014, 'people'),
   (0.010919612, 'right'),
   (0.010300203, 'fifa'),
   (0.009747476, 'can')],
  -2.178397379771609),
 ([(0.011165289, 'like'),
   (0.009153973, 'just'),
   (0.008411711, 'qatar'),
   (0.0073926575, 'll'),
   (0.007355442, 'go')],
  -2.650815749969305),
 ([(0.015905043, 'goal'),
   (0.014290159, 'qatar'),
   (0.013865782, 'will'),
   (0.0114682475, 'offside'),
   (0.011273708, 'match')],
  -2.8277855405941352),
 ([(0.013977946, 'qatar'),
   (0.012350101, 'iran'),
   (0.011916313, 'group'),
   (0.008455997, 'england'),
   (0.008005491, 'fifa')],
  -2.972876181639534)]


In [15]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.018144 -0.099592       1        1  27.757063
4      0.149344 -0.057956       2        1  25.346279
1     -0.026409 -0.067478       3        1  18.111775
0     -0.202269  0.056277       4        1  15.563394
3      0.097478  0.168749       5        1  13.221489, topic_info=         Term        Freq       Total Category  logprob  loglift
1432     goal  183.000000  183.000000  Default  30.0000  30.0000
130       fan  339.000000  339.000000  Default  29.0000  29.0000
1064    group  193.000000  193.000000  Default  28.0000  28.0000
503      iran  173.000000  173.000000  Default  27.0000  27.0000
1552  offside  110.000000  110.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
289       see   32.970448  229.309748   Topic5  -5.5084   0.0839
676      well   27.919343  160.444157   Topic5  -5.6747   0.2747
155   thought   23.702098   61.180503   Topic5  -5.8384   1.0751
50       even   26.711791  272.785151   Topic5  -5.7189  -0.3003
40      thing   24.544923  130.513875   Topic5  -5.8035   0.3524

[372 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
912       3  0.883204  absolute
912       5  0.103906  absolute
2851      3  0.961308   account
492       2  0.934089    acting
4106      5  0.982257     ahead
...     ...       ...       ...
174       4  0.087047      year
1679      5  0.978997    yellow
1862      5  0.908220     youtu
3027      3  0.962910   youtube
4764      5  0.978599      zero

[606 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 2, 1, 4])

In [ ]:
print(ldamodel.get_document_topics(corpus))

In [ ]:
top_topics = []
for i, row in enumerate(ldamodel[corpus]):
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    top_topics.append(row)

In [ ]:
len(top_topics)

In [ ]:
top_topics[5]

In [20]:
doc_list[10]

'Nah. Pay FIFA enough money you can do whatever you want. They can have World Cups in Arab countries every year if they keep paying the corrupt FIFA organization.'

/opt/conda/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [5]:
len(doc_list)

5599